# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:

main_df = pd.read_csv("Output/Cities.csv", encoding="utf-8")
main_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hong Kong,22.29,114.16,298.15,83,1,2.70,HK,1605831620
1,Bredasdorp,-34.53,20.04,285.15,87,6,1.50,ZA,1605831697
2,Saint Paul Harbor,57.79,-152.41,276.15,93,90,4.08,US,1605831488
3,San Patricio,28.02,-97.52,296.15,64,20,4.60,US,1605831698
4,Alghero,40.56,8.32,289.15,59,0,4.60,IT,1605831574
...,...,...,...,...,...,...,...,...,...
563,Chișinău,47.01,28.86,277.15,80,90,2.60,MD,1605831834
564,Kedgwick,47.64,-67.34,267.46,97,100,4.63,CA,1605831979
565,Riyadh,24.69,46.72,292.15,48,0,2.60,SA,1605831969
566,Mareeba,-17.00,145.43,302.15,51,40,8.70,AU,1605831980


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [7]:
# Store latitude and longitude in locations
locations = main_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = main_df["Humidity"].astype(float)
maxhumidity = humidity.max()


In [8]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, point_radius=3, max_intensity=maxhumidity)



# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
fav_city_df = main_df.loc[(main_df["Max Temp"] > 250) & (main_df["Max Temp"] < 350) & (main_df["Cloudiness"] == 1), :]

fav_city_df = fav_city_df.dropna(how='any')

fav_city_df.reset_index(inplace=True)

del fav_city_df['index']

fav_city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hong Kong,22.29,114.16,298.15,83,1,2.70,HK,1605831620
1,Jamestown,42.10,-79.24,286.15,30,1,7.70,US,1605831700
2,Laguna,38.42,-121.42,289.15,55,1,5.70,US,1605831707
3,Saint George,37.10,-113.58,294.15,21,1,0.57,US,1605831710
4,Bethel,41.37,-73.41,279.15,48,1,4.10,US,1605831714


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotels = []

for i in range(len(fav_city_df)):
    lat = fav_city_df.loc[i]['Lat']
    lng = fav_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
fav_city_df["Hotel Name"] = hotels
fav_city_df = fav_city_df.dropna(how='any')
fav_city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Hong Kong,22.29,114.16,298.15,83,1,2.70,HK,1605831620,Hong Kong
1,Jamestown,42.10,-79.24,286.15,30,1,7.70,US,1605831700,Jamestown
2,Laguna,38.42,-121.42,289.15,55,1,5.70,US,1605831707,Sacramento
3,Saint George,37.10,-113.58,294.15,21,1,0.57,US,1605831710,St. George
4,Bethel,41.37,-73.41,279.15,48,1,4.10,US,1605831714,Danbury


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in fav_city_df.iterrows()]
locations = fav_city_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure
